In [393]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [394]:
DIR_PATH = '/Users/carlosperezricardo/Documents/data/'
commercial_activity = pd.read_csv(DIR_PATH+'commercial_activity_df.csv', encoding='utf-8')
commercial_activity.drop(columns=['Unnamed: 0'], inplace=True)

products = pd.read_csv(DIR_PATH+'products_df.csv', encoding='utf-8')
products.drop(columns=['Unnamed: 0'], inplace=True)

sociodemographic = pd.read_csv(DIR_PATH+'sociodemographic_df.csv', encoding='utf-8')
sociodemographic.drop(columns=['Unnamed: 0'], inplace=True)

In [395]:
df_whole = pd.merge( commercial_activity, products, on = ['pk_cid','pk_partition'] )
df_whole = pd.merge( df_whole, sociodemographic, on=['pk_cid','pk_partition'] )

In [396]:
df_whole.shape

(5962924, 27)

In [397]:
df_whole.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Columns: 27 entries, pk_cid to salary
dtypes: float64(5), int64(15), object(7)
memory usage: 1.2+ GB


In [398]:
# We reduce memory
# Codificar variables logicas (1/0 = bool)
from sys import getsizeof

boolean_cols = ["short_term_deposit", "loans", "mortgage", "funds", "securities","long_term_deposit", "em_account_pp", "credit_card", "payroll_account", "emc_account", "debit_card", "em_account_p", "em_acount", "payroll", "pension_plan"] 
# payroll y pension_plan tienen nulos en el dataset completo

print('El dataframe products ocupa {} MB antes de codificar columnas boolean'.format(getsizeof(df_whole)/1048576))

for x in boolean_cols:
    df_whole[x] = df_whole[x].astype(bool)

print('El dataframe products_df ocupa {} MB después de codificar columnas boolean'.format(getsizeof(df_whole)/1048576))

El dataframe products ocupa 3468.526909828186 MB antes de codificar columnas boolean
El dataframe products_df ocupa 2871.4247255325317 MB después de codificar columnas boolean


In [399]:
df_whole.head(3).T

,0,1,2
pk_cid,1375586,1050611,1050612
pk_partition,2018-01-28,2018-01-28,2018-01-28
entry_date,2018-01-12,2015-08-10,2015-08-10
entry_channel,KHL,KHE,KHE
active_customer,1,0,0
segment,02 - PARTICULARES,03 - UNIVERSITARIO,03 - UNIVERSITARIO
short_term_deposit,False,False,False
loans,False,False,False
mortgage,False,False,False
funds,False,False,False


In [400]:
df_whole.columns.to_list()

['pk_cid',
 'pk_partition',
 'entry_date',
 'entry_channel',
 'active_customer',
 'segment',
 'short_term_deposit',
 'loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount',
 'country_id',
 'region_code',
 'gender',
 'age',
 'deceased',
 'salary']

In [401]:
df = pd.DataFrame()

# Salary (1)

In [402]:
salary_check = df_whole.groupby('pk_cid')['salary'].agg(['min','max'])
salary_check = salary_check.dropna(how='any')
salary_check[ salary_check['min'] != salary_check['max'] ]

# There is no change in salary

,min,max
pk_cid,,


In [403]:
salary_check['max'] = salary_check['max'].astype('int')
salary_check['min'] = salary_check['min'].astype('int')
salary_check.head()

,min,max
pk_cid,,
17457,102405,102405
17590,407281,407281
17799,84511,84511
17980,282461,282461
18704,75187,75187


In [404]:
salary_check['max'].median()

89018.0

In [405]:
df['salary'] = df_whole.groupby('pk_cid')['salary'].max()

# Muertos (1)

In [406]:
df_whole.groupby('pk_cid')['deceased'].max().value_counts()

N    456244
S       129
Name: deceased, dtype: int64

In [407]:
df_whole['deceased'] = df_whole['deceased'].replace({'N':0,'S':1})

In [408]:
df['deceased'] = df_whole.groupby('pk_cid')['deceased'].max()

# Cliente activo (1)

In [409]:
df['active_customer'] = df_whole.groupby('pk_cid')['active_customer'].last()

# Edad (1)

Nos quedamos con la ultima o con la edad más grande.

In [410]:
df_whole.groupby('pk_cid')['age'].max()

pk_cid
15891      59
16063      62
16203      70
16502      58
17457      54
           ..
1553685    52
1553686    30
1553687    21
1553688    43
1553689    40
Name: age, Length: 456373, dtype: int64

In [411]:
df['age'] = df_whole.groupby('pk_cid')['age'].max()

# Genero 

No la utilizamos.

# Region code (2)

Pasamos a Provincia. Podemos meter población en esa provincia. O de tu población de productos.
Frequency encoding.

Total % de tu cartera que son de Madrid 
Total % que de Madrid tiene productos sobre el total. Que tanto % de clientes de Madrid compran.

In [412]:
top_regions = df_whole.groupby('pk_cid')['region_code'].last().to_frame().value_counts().head(5)
list_top_regions = top_regions.reset_index()['region_code'].to_list()
top_regions
list_top_regions

[28.0, 8.0, 46.0, 30.0, 41.0]

In [413]:
df_whole['region_code'] = np.where( df_whole['region_code'].isin( list_top_regions ), True, False )

In [414]:
df_whole.groupby('pk_cid')['region_code'].last().value_counts()

False    241199
True     215174
Name: region_code, dtype: int64

In [415]:
df['region_code'] = df_whole.groupby('pk_cid')['region_code'].last()

# country_id (1)

Agrupar por Continente y hacer lo mismo. Pasar a no_spain.

In [416]:
df_whole['country_id'] = np.where( df_whole['country_id'] == 'ES', True, False )

In [417]:
df_whole.groupby('pk_cid')['country_id'].last().value_counts()

True     456209
False       164
Name: country_id, dtype: int64

In [418]:
df['country_id'] = df_whole.groupby('pk_cid')['country_id'].last()

# payroll (1)

In [419]:
df_whole['payroll'] = np.where( df_whole['country_id'] == 'ES', True, False )

In [420]:
df_whole.groupby('pk_cid')['payroll'].last().value_counts()

False    456373
Name: payroll, dtype: int64

In [421]:
df['payroll'] = df_whole.groupby('pk_cid')['payroll'].last()

# Por producto 

- cantidad de altas (17)
- cantidad de bajas sin cobro / altas que no han sido cobradas (17)
- cantidad de bajas con cobro (17)

# Por tipo de producto

- Cantidad de altas (3)
- Cantidad de bajas (3)

# Customer Lifetime Value

- Gastos totales de cada cliente (1)
- Gastos totales en cada tipo de producto (3)
- Numero de productos contratados en su historia (1)



In [422]:
customers = df.index
len(customers)

456373

In [423]:
partitions = ['2018-01-28','2018-02-28','2018-03-28','2018-04-28','2018-05-28','2018-06-28', \
    '2018-07-28','2018-08-28','2018-09-28','2018-10-28','2018-11-28','2018-12-28','2019-01-28', \
        '2019-02-28','2019-03-28','2019-04-28','2019-05-28']

list_products = ['short_term_deposit','loans','mortgage','funds','securities',
    'long_term_deposit','em_account_pp','credit_card','pension_plan',
    'payroll_account','emc_account','debit_card','em_account_p','em_acount']

products_dict = {"short_term_deposit":"ahorro e inversión", "loans":"financiación", "mortgage":"financiación", 
    "funds":"ahorro e inversión", "securities":"ahorro e inversión", "long_term_deposit":"ahorro e inversión", 
    "em_account_pp":"cuenta", "credit_card":"financiación", "payroll_account":"cuenta", "pension_plan":"ahorro e inversión", 
    "emc_account":"cuenta", "debit_card":"financiación", "em_account_p":"cuenta", "em_acount":"cuenta"}

cost_product = {'cuenta':10, 'ahorro e inversión':40, 'financiación':60}

In [424]:
def determinar_altas(data):
    data = pd.DataFrame(data)
    data.columns=['product']
    data['prev'] = data['product'].shift(1)
    data['diff'] = data['product'] - data['prev']

    return len(data[data['diff'] == 1])

In [425]:
test_data = pd.DataFrame( [0,0,1,1,1,0,0,0,1,1,1,1,0,0,0,1] )
determinar_altas(test_data)
# expected result: 3

3

In [426]:
def determinar_bajas(data):
    data = pd.DataFrame(data)
    data.columns=['product']
    data['prev'] = data['product'].shift(1)
    data['diff'] = data['product'] - data['prev']

    return len(data[data['diff'] == -1])

In [427]:
test_data = pd.DataFrame( [0,0,1,1,1,0,0,0,1,1,1,1,0,0,0,1,0] )
determinar_bajas(test_data)
# expected result: 2

3

In [428]:
def determinar_altas_all(data):
    data.columns=['pk_partition','pk_cid','product']
    data['prev'] = data.groupby('pk_cid')['product'].shift(1)
    data['diff'] = data['product'] - data['prev']

    # Solo queremos 1 
    data['diff'] = np.where( (data['product']==1) & (data['diff'].isna()) & (data['pk_partition']!='2018-01-28'), 1, data['diff'] )
    data['diff'].fillna(0,inplace=True)
    data['diff'] = np.where( data['diff'] == -1, 0, data['diff'] )

    return data.groupby('pk_cid')['diff'].sum()

In [429]:
def determinar_bajas_all(data):
    data.columns=['pk_cid','product']
    data['prev'] = data.groupby('pk_cid')['product'].shift(1)
    data['diff'] = data['product'] - data['prev']

    # Solo queremos -1 
    data['diff'].fillna(0,inplace=True)
    data['diff'] = np.where( data['diff'] == 1, 0, data['diff'] )
    data['diff'] = np.where( data['diff'] == -1, 1, data['diff'] )

    return data.groupby('pk_cid')['diff'].sum()

In [430]:
def determinar_altas_cobradas_all(data):
    data.columns=['pk_partition','pk_cid','product']

    data['prev_month1'] = data.groupby('pk_cid')['product'].shift(1)
    data['prev_month2'] = data.groupby('pk_cid')['product'].shift(2)
    data['prev_month3'] = data.groupby('pk_cid')['product'].shift(3)

    data.fillna(-2, inplace=True)

    # nuevas contrataciones (ya cobrados - llevan 3 meses y no se han dado de baja)
    #cond3 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) &\
    #    ((data['prev_month3']==0) | ((data['prev_month3'] == -2) & (data['pk_partition']!='2018-03-28')))

    cond31 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==0)
    cond32 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==-2) & (data['pk_partition']!='2018-03-28')
    cond33 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==1) & (data['pk_partition']=='2018-03-28')
    
    cond3 = cond31 | cond32 | cond33

    data['comprado'] = cond3

    return data.groupby('pk_cid')['comprado'].sum()

In [431]:
df_whole = df_whole.sort_values(['pk_partition','pk_cid'], ascending=True)

In [432]:
for prod in list_products:
    df[prod+'_altas'] = determinar_altas_all( df_whole[['pk_partition','pk_cid',prod]] )
    df[prod+'_bajas'] = determinar_bajas_all( df_whole[['pk_cid',prod]] )
    df[prod+'_cobros'] = determinar_altas_cobradas_all( df_whole[['pk_partition','pk_cid',prod]] )
    print(prod)

short_term_deposit
loans
mortgage
funds
securities
long_term_deposit
em_account_pp
credit_card
pension_plan
payroll_account
emc_account
debit_card
em_account_p
em_acount


In [433]:
df.head(3).T

pk_cid,15891,16063,16203
salary,NaN,NaN,NaN
deceased,0,0,0
active_customer,0,0,1
age,59,62,70
region_code,True,True,True
country_id,True,True,True
payroll,False,False,False
short_term_deposit_altas,0,0,0
short_term_deposit_bajas,0,0,0
short_term_deposit_cobros,0,0,0


In [436]:
for key, value in products_dict.items():
    # corregimos titulos largos y con tildes
    if value == 'ahorro e inversión':
        value = 'ahorros'
    elif value == 'financiación':
        value = 'financiacion'
    elif value == 'cuenta':
        value = 'cuenta'

    if value in df.columns:
        df[value+'_altas'] += df[key+'_altas'] 
        df[value+'_bajas'] += df[key+'_bajas'] 
        df[value+'_cobros'] += df[key+'_cobros'] 

    else:
        df[value+'_altas'] = df[key+'_altas']
        df[value+'_bajas'] = df[key+'_bajas'] 
        df[value+'_cobros'] = df[key+'_cobros'] 

    #df.drop(key+'_altas', axis=1, inplace=True)
    #df.drop(key+'_bajas', axis=1, inplace=True)
    #df.drop(key+'_cobros', axis=1, inplace=True)

In [437]:
df.columns

Index(['salary', 'deceased', 'active_customer', 'age', 'region_code',
       'country_id', 'payroll', 'short_term_deposit_altas',
       'short_term_deposit_bajas', 'short_term_deposit_cobros', 'loans_altas',
       'loans_bajas', 'loans_cobros', 'mortgage_altas', 'mortgage_bajas',
       'mortgage_cobros', 'funds_altas', 'funds_bajas', 'funds_cobros',
       'securities_altas', 'securities_bajas', 'securities_cobros',
       'long_term_deposit_altas', 'long_term_deposit_bajas',
       'long_term_deposit_cobros', 'em_account_pp_altas',
       'em_account_pp_bajas', 'em_account_pp_cobros', 'credit_card_altas',
       'credit_card_bajas', 'credit_card_cobros', 'pension_plan_altas',
       'pension_plan_bajas', 'pension_plan_cobros', 'payroll_account_altas',
       'payroll_account_bajas', 'payroll_account_cobros', 'emc_account_altas',
       'emc_account_bajas', 'emc_account_cobros', 'debit_card_altas',
       'debit_card_bajas', 'debit_card_cobros', 'em_account_p_altas',
       'em_acco

# Customer Lifetime Value

In [441]:
for key, value in cost_product.items():
    if key == 'ahorro e inversión':
        key = 'ahorros'
    elif key == 'financiación':
        key = 'financiacion'
    elif key == 'cuenta':
        key = 'cuenta'

    if 'gastado' not in df.columns:
        df['gastado'] = df[key+'_cobros']*value
    else:
        df['gastado'] += df[key+'_cobros']*value

    #df['bajas'] += df[key+'_bajas']
    #df.drop(key+'_bajas', axis=1, inplace=True)
    #df.drop(key+'_cobros', axis=1, inplace=True)

In [442]:
df.head(3).T

pk_cid,15891,16063,16203
salary,NaN,NaN,NaN
deceased,0,0,0
active_customer,0,0,1
age,59,62,70
region_code,True,True,True
country_id,True,True,True
payroll,False,False,False
short_term_deposit_altas,0,0,0
short_term_deposit_bajas,0,0,0
short_term_deposit_cobros,0,0,0


In [443]:
len(df.columns)

59

In [444]:
products_dict = {"short_term_deposit":"ahorro e inversión", "loans":"financiación", "mortgage":"financiación", 
    "funds":"ahorro e inversión", "securities":"ahorro e inversión", "long_term_deposit":"ahorro e inversión", 
    "em_account_pp":"cuenta", "credit_card":"financiación", "payroll_account":"cuenta", "pension_plan":"ahorro e inversión", 
    "emc_account":"cuenta", "debit_card":"financiación", "em_account_p":"cuenta", "em_acount":"cuenta"}

ahorros = ["pk_partition","short_term_deposit","funds","securities","long_term_deposit","pension_plan"]
cuentas = ["pk_partition","em_acount","em_account_pp","emc_account","em_account_p","em_account_pp","payroll_account"]
financiacion = ["pk_partition","short_term_deposit","funds","securities","long_term_deposit","pension_plan"]

In [445]:
df_whole[ df_whole['pk_cid'] == 16203 ][cuentas]
# cuentas: altas = 1, cobradas = 1, bajas = 0

,pk_partition,em_acount,em_account_pp,emc_account,em_account_p,em_account_pp,payroll_account
3628238,2018-12-28,False,False,False,False,False,False
4028170,2019-01-28,True,False,False,False,False,False
4480636,2019-02-28,True,False,False,False,False,False
4876039,2019-03-28,True,False,False,False,False,False
5134321,2019-04-28,True,False,False,False,False,False
5523396,2019-05-28,True,False,False,False,False,False


In [446]:
df_whole[ df_whole['pk_cid'] == 15891 ][cuentas]
# cuentas: altas = 1, cobradas = 0, bajas = 1

,pk_partition,em_acount,em_account_pp,emc_account,em_account_p,em_account_pp,payroll_account
1479563,2018-07-28,True,False,False,False,False,False
2168122,2018-08-28,False,False,False,False,False,False


In [447]:
def determinar_altas_cobradas(data):
    data = pd.DataFrame(data)
    data.reset_index(inplace=True)

    data.columns=['pk_partition','product']
    
    data['prev_month1'] = data['product'].shift(1)
    data['prev_month2'] = data['product'].shift(2)
    data['prev_month3'] = data['product'].shift(3)

    data.fillna(-2, inplace=True)

    # nuevas contrataciones (ya cobrados - llevan 3 meses y no se han dado de baja)
    #cond3 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) &\
    #    ((data['prev_month3']==0) | ((data['prev_month3'] == -2) & (data['pk_partition']!='2018-03-28')))
    
    cond1 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==0)
    cond2 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==-2) & (data['pk_partition']!='2018-03-28')

    cond3 = cond1 | cond2
    print(data)
    contrataciones_cobradas = data[ cond3 ] 

    return len(contrataciones_cobradas)

In [448]:
def determinar_altas_cobradas_all(data):
    data = pd.DataFrame(data)
    data.reset_index(inplace=True)

    data.columns=['pk_partition','product']
    
    data['prev_month1'] = data['product'].groupby('pk_cid').shift(1)
    data['prev_month2'] = data['product'].groupby('pk_cid').shift(2)
    data['prev_month3'] = data['product'].groupby('pk_cid').shift(3)

    data.fillna(-2, inplace=True)

    # nuevas contrataciones (ya cobrados - llevan 3 meses y no se han dado de baja)
    #cond3 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) &\
    #    ((data['prev_month3']==0) | ((data['prev_month3'] == -2) & (data['pk_partition']!='2018-03-28')))
    
    cond1 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==0)
    cond2 = (data['product']==1) & (data['prev_month1']==1) & (data['prev_month2']==1) & (data['prev_month3']==-2) & (data['pk_partition']!='2018-03-28')

    cond3 = cond1 | cond2

    data['comprado'] = cond3
    print(data)
    contrataciones_cobradas = data[ cond3 ] 

    #return len(contrataciones_cobradas)

In [449]:
test_data = pd.DataFrame( [0,0,1,1,1,1,0,0,1,1,0,0,0,0,0,1,1], index=partitions )
test_data.reset_index(inplace=True)
test_data.columns = ['pk_partition','product']
test_data['pk_partition'] = pd.to_datetime( test_data['pk_partition'] )
test_data.set_index('pk_partition',inplace=True)

determinar_altas_cobradas(test_data)
# expected result: 1

   pk_partition  product  prev_month1  prev_month2  prev_month3
0    2018-01-28        0         -2.0         -2.0         -2.0
1    2018-02-28        0          0.0         -2.0         -2.0
2    2018-03-28        1          0.0          0.0         -2.0
3    2018-04-28        1          1.0          0.0          0.0
4    2018-05-28        1          1.0          1.0          0.0
5    2018-06-28        1          1.0          1.0          1.0
6    2018-07-28        0          1.0          1.0          1.0
7    2018-08-28        0          0.0          1.0          1.0
8    2018-09-28        1          0.0          0.0          1.0
9    2018-10-28        1          1.0          0.0          0.0
10   2018-11-28        0          1.0          1.0          0.0
11   2018-12-28        0          0.0          1.0          1.0
12   2019-01-28        0          0.0          0.0          1.0
13   2019-02-28        0          0.0          0.0          0.0
14   2019-03-28        0          0.0   

1

In [450]:
test_data = pd.DataFrame( [1,1,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1], index=partitions )
test_data.reset_index(inplace=True)
test_data.columns = ['pk_partition','product']
test_data['pk_partition'] = pd.to_datetime( test_data['pk_partition'] )
test_data.set_index('pk_partition',inplace=True)

determinar_altas_cobradas(test_data)
# expected result: 2

   pk_partition  product  prev_month1  prev_month2  prev_month3
0    2018-01-28        1         -2.0         -2.0         -2.0
1    2018-02-28        1          1.0         -2.0         -2.0
2    2018-03-28        1          1.0          1.0         -2.0
3    2018-04-28        1          1.0          1.0          1.0
4    2018-05-28        0          1.0          1.0          1.0
5    2018-06-28        1          0.0          1.0          1.0
6    2018-07-28        0          1.0          0.0          1.0
7    2018-08-28        0          0.0          1.0          0.0
8    2018-09-28        1          0.0          0.0          1.0
9    2018-10-28        1          1.0          0.0          0.0
10   2018-11-28        1          1.0          1.0          0.0
11   2018-12-28        1          1.0          1.0          1.0
12   2019-01-28        0          1.0          1.0          1.0
13   2019-02-28        0          0.0          1.0          1.0
14   2019-03-28        1          0.0   

2

In [451]:
test_data = pd.DataFrame( [1,1,1,1,0,1,0,0,1,1,1,1,0], index=partitions[4:] )
test_data.reset_index(inplace=True)
test_data.columns = ['pk_partition','product']
test_data['pk_partition'] = pd.to_datetime( test_data['pk_partition'] )
test_data.set_index('pk_partition',inplace=True)

determinar_altas_cobradas(test_data)
# expected result: 2

   pk_partition  product  prev_month1  prev_month2  prev_month3
0    2018-05-28        1         -2.0         -2.0         -2.0
1    2018-06-28        1          1.0         -2.0         -2.0
2    2018-07-28        1          1.0          1.0         -2.0
3    2018-08-28        1          1.0          1.0          1.0
4    2018-09-28        0          1.0          1.0          1.0
5    2018-10-28        1          0.0          1.0          1.0
6    2018-11-28        0          1.0          0.0          1.0
7    2018-12-28        0          0.0          1.0          0.0
8    2019-01-28        1          0.0          0.0          1.0
9    2019-02-28        1          1.0          0.0          0.0
10   2019-03-28        1          1.0          1.0          0.0
11   2019-04-28        1          1.0          1.0          1.0
12   2019-05-28        0          1.0          1.0          1.0


2

# Permanencia (1)

Tenemos muchos productos. Como mido la permanencia sin tener 20 columnas. 

Permanencia = sí y solo sí en ese producto se ha comprado se tiene en cuenta

perm = sumatorio de num.meses / veces que lo contrato

In [452]:
def determinar_permanencia(data):
    data = pd.DataFrame(data)
    data.reset_index(inplace=True)

    data.columns=['pk_partition','product']

    data['diff'] = data['product'].diff()

    data['prev'] = np.where( (data['diff'].isna()) & (data['product'] == 1), 1, 0 )
    data = data[ data['diff'] != 0 ]

    data['date_diff'] = round(data['pk_partition'].diff()/np.timedelta64(1, 'M'))

    return data[data['diff'] == -1]

In [453]:
def determinar_permanencia_all(data):
    data.columns=['pk_partition','pk_cid','product']
    data['pk_partition'] = pd.to_datetime(data['pk_partition'])

    data['prev'] = data.groupby('pk_cid')['product'].shift(1)
    data['diff'] = data['product'] - data['prev']
    #data['diff'] = data['product'].diff()

    data['prev'] = np.where( (data['diff'].isna()) & (data['product'] == 1), 1, 0 )
    data = data[ data['diff'] != 0 ]
    
    data = data.sort_values(['pk_cid','pk_partition'])
    
    data['date_diff'] = round(data['pk_partition'].diff()/np.timedelta64(1, 'M'))

    data = data[data['diff'] == -1]
    
    #print(data)
    summary = data.groupby('pk_cid')['date_diff'].agg(['sum','count'])
    summary['result'] = summary['sum']/summary['count']
    summary.reset_index(inplace=True)
    #print(summary)

    return summary[['pk_cid','result']]

In [454]:
list_perms = []
for prod in list_products:
    perms = determinar_permanencia_all( df_whole[['pk_partition','pk_cid',prod]] )
    #print(perms)
    perms.columns = ['pk_cid',prod+'_perm']
    df = pd.merge(df, perms, how='left', on='pk_cid')
    list_perms.append(prod+'_perm')
    print(prod)

short_term_deposit
loans
mortgage
funds
securities
long_term_deposit
em_account_pp
credit_card
pension_plan
payroll_account
emc_account
debit_card
em_account_p
em_acount


In [455]:
df.head(3).T

,0,1,2
pk_cid,15891,16063,16203
salary,NaN,NaN,NaN
deceased,0,0,0
active_customer,0,0,1
age,59,62,70
...,...,...,...
payroll_account_perm,NaN,NaN,NaN
emc_account_perm,NaN,NaN,NaN
debit_card_perm,NaN,NaN,NaN
em_account_p_perm,NaN,NaN,NaN


In [456]:
df[list_perms].sum(axis=1)/df[list_perms].count(axis=1)

0         1.0
1         NaN
2         NaN
3         NaN
4         5.0
         ... 
456368    NaN
456369    NaN
456370    NaN
456371    NaN
456372    NaN
Length: 456373, dtype: float64

In [457]:
df['permanencia'] = df[list_perms].sum(axis=1)/df[list_perms].count(axis=1)
df['permanencia'].value_counts(dropna=False)

NaN         375200
1.000000     14892
2.000000      8109
3.000000      7097
4.000000      4254
             ...  
1.083333         1
8.888889         1
2.088889         1
9.625000         1
8.142857         1
Name: permanencia, Length: 675, dtype: int64

In [458]:
df['permanencia'].fillna(0, inplace=True)

In [459]:
df.drop(list_perms, axis=1, inplace=True)

In [460]:
prod = 'short_term_deposit'
df_whole[ df_whole['pk_cid'] == 209630 ][['pk_cid','pk_partition',prod]]

,pk_cid,pk_partition,short_term_deposit
238092,209630,2018-01-28,True
241000,209630,2018-02-28,False
483568,209630,2018-03-28,False
1479387,209630,2018-07-28,False
2167899,209630,2018-08-28,False
2172146,209630,2018-09-28,False
2943809,209630,2018-10-28,False
3033558,209630,2018-11-28,False
3627510,209630,2018-12-28,False
3839899,209630,2019-01-28,False


In [461]:
test_data = pd.DataFrame( [0,0,1,1,1,0,0,0,1,1,1,1,0,0,0,1,1], index=partitions )
test_data.reset_index(inplace=True)
test_data.columns = ['pk_partition','product']
test_data['pk_partition'] = pd.to_datetime( test_data['pk_partition'] )
test_data.set_index('pk_partition',inplace=True)

perm = determinar_permanencia(test_data)
print(perm)

perm['date_diff'].sum()/len(perm['date_diff'])
# expected result: 3, 4

   pk_partition  product  diff  prev  date_diff
5    2018-06-28        0  -1.0     0        3.0
12   2019-01-28        0  -1.0     0        4.0


3.5

In [462]:
#for prod in list_products:
    #df[prod+'_altas'] = df_whole[['pk_cid',prod]].agg(determinar_altas_cobradas_all)

In [ ]:
#for prod in list_products:
#    df[prod+'_altas'] = df_whole.groupby('pk_cid')[prod].agg(determinar_altas)
#    print(prod)

In [ ]:
#for prod in list_products:
#    df[prod+'_bajas'] = df_whole.groupby('pk_cid')[prod].agg(determinar_bajas)

In [ ]:
#for prod in list_products:
#    df[prod+'_cobros'] = df_whole.groupby('pk_cid')[prod].agg(determinar_altas_cobradas)

## Numero de productos contratados en su historia (1)

In [463]:
df['total_products'] = df_whole.groupby('pk_cid')[list_products].max().sum(axis=1).reset_index(drop=True)

In [464]:
df.head(3).T

,0,1,2
pk_cid,15891,16063,16203
salary,NaN,NaN,NaN
deceased,0,0,0
active_customer,0,0,1
age,59,62,70
...,...,...,...
cuenta_bajas,1,0,0
cuenta_cobros,0,0,1
gastado,0,0,40
permanencia,1,0,0


In [ ]:
df['bajas'] = df['ahorros_bajas'] + df['financiacion_bajas'] + df['cuenta_bajas'] 

In [ ]:
len(df.columns)

In [465]:
df['payroll'] = df['payroll'].astype(int)

In [466]:
# drop de columnas de altas, bajas y cobros por cada producto
for key, value in products_dict.items():
    df.drop(key+'_altas', axis=1, inplace=True)
    df.drop(key+'_bajas', axis=1, inplace=True)
    df.drop(key+'_cobros', axis=1, inplace=True)

In [472]:
show_columns = df.columns.to_list()
show_columns.remove('pk_cid')
show_columns.remove('payroll')

show_columns.remove('ahorros_bajas')
show_columns.remove('financiacion_bajas')
show_columns.remove('cuenta_bajas')

show_columns.remove('ahorros_altas')
show_columns.remove('financiacion_altas')
show_columns.remove('cuenta_altas')

print(len(show_columns))

corr = df[show_columns].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

13


,salary,deceased,active_customer,age,region_code,country_id,ahorros_cobros,financiacion_cobros,cuenta_cobros,gastado,permanencia,total_products,bajas
salary,1.000,0.001,0.002,0.011,0.111,-0.001,-0.003,-0.004,-0.008,-0.008,0.003,-0.008,0.001
deceased,0.001,1.000,0.003,0.053,0.003,-0.007,0.004,0.003,-0.001,0.003,0.031,0.018,0.017
active_customer,0.002,0.003,1.000,0.228,-0.024,-0.010,0.216,0.296,0.243,0.398,0.230,0.547,0.293
age,0.011,0.053,0.228,1.000,0.059,-0.017,0.092,0.104,-0.007,0.088,0.170,0.233,0.135
region_code,0.111,0.003,-0.024,0.059,1.000,0.018,0.030,0.008,-0.036,-0.008,0.023,-0.017,0.019
country_id,-0.001,-0.007,-0.010,-0.017,0.018,1.000,0.003,-0.001,0.002,0.002,-0.004,-0.007,-0.002
ahorros_cobros,-0.003,0.004,0.216,0.092,0.030,0.003,1.000,0.205,-0.039,0.434,0.274,0.465,0.437
financiacion_cobros,-0.004,0.003,0.296,0.104,0.008,-0.001,0.205,1.000,0.126,0.738,0.255,0.405,0.414
cuenta_cobros,-0.008,-0.001,0.243,-0.007,-0.036,0.002,-0.039,0.126,1.000,0.678,0.074,0.196,0.075
gastado,-0.008,0.003,0.398,0.088,-0.008,0.002,0.434,0.738,0.678,1.000,0.288,0.517,0.437


In [473]:
df.columns

Index(['pk_cid', 'salary', 'deceased', 'active_customer', 'age', 'region_code',
       'country_id', 'payroll', 'ahorros_altas', 'ahorros_bajas',
       'ahorros_cobros', 'financiacion_altas', 'financiacion_bajas',
       'financiacion_cobros', 'cuenta_altas', 'cuenta_bajas', 'cuenta_cobros',
       'gastado', 'permanencia', 'total_products', 'bajas'],
      dtype='object')

In [ ]:
#prod = list_products[0]

#df[prod+'_altas'] = df_whole.groupby('pk_cid')[prod].agg(determinar_altas)
#df[prod+'_bajas'] = df_whole.groupby('pk_cid')[prod].agg(determinar_bajas)

In [ ]:
df.head()

,pk_cid,salary,deceased,active_customer,age,region_code,country_id,payroll,ahorro e inversión_altas,ahorro e inversión_bajas,ahorro e inversión_cobros,financiación_altas,financiación_bajas,financiación_cobros,cuenta_altas,cuenta_bajas,cuenta_cobros,gastado,permanencia,total_products
0,15891,NaN,0,0.0,59,True,True,0,0,0,0,0,0,0,1,1,0,0,1.0,1
1,16063,NaN,0,0.0,62,True,True,0,0,0,0,0,0,0,0,0,0,0,0.0,0
2,16203,NaN,0,1.0,70,True,True,0,0,0,0,0,0,0,1,0,1,10,0.0,1
3,16502,NaN,0,1.0,58,True,True,0,0,0,0,0,0,0,1,0,1,10,0.0,2
4,17457,102405.75,0,1.0,54,True,True,0,0,0,0,5,4,0,0,0,0,0,5.0,3
